The theory for this notebook is developed in `ZonotopesNonlinearReach`.

## Dependencies

In [ ]:
# load packages
using Plots
using LazySets, MathematicalSystems, Reachability
using LazySets.Approximations
using Reachability: center
using Reachability.ReachSets: Φ₁
using IntervalArithmetic, ValidatedNumerics
using LazySets: Interval, translate
using TaylorSeries
using TaylorSeries: gradient, jacobian, hessian, derivative
const ∂ = derivative
using RecursiveArrayTools

After loading the packages, you should precompile the functions in the following section.

## Model

In [ ]:
include("ZonotopesNonlinearReach/circle.jl")
@time begin 𝑃, 𝑂, 𝑂_ASB08 = circle() end;

# include("ZonotopesNonlinearReach/vanderpol.jl")
# @time begin 𝑃, 𝑂, 𝑂_ASB08 = vanderpol() end;

# include("ZonotopesNonlinearReach/laubloomis.jl")
# @time begin 𝑃, 𝑂, 𝑂_ASB08 = laubloomis() end;

In [ ]:
𝑃.s.f

## Functions for sound linearization

In [ ]:
# f    -- nonlinear ODE
# 𝑋₀i  -- initial set of current chunk
# n    -- state dimension
# δ    -- step size
function linearize(f, n, 𝑋₀i, δ)
    # linearization point for current chunk
    c = center(𝑋₀i)
    x̃ = c + δ/2 * f(c)

    # evaluate Jacobian at the linearization point
    Ax̃ = jacobian(f, x̃) #  map(x -> evaluate(x, x̃), Jf)
    bx̃ = f(x̃) - Ax̃ * x̃

    # instantiate linearized system; it doesn't have state constraints
    𝑆lin = CACS(Ax̃, bx̃, Universe(n)) # shall we accept AffineContSys and normalize?
    𝑃lin = IVP(𝑆lin, 𝑋₀i)
    return x̃, 𝑃lin
end

# let's assume n = 2 to simplify
function admissible_error(Ax̃, δ, θ; n=2)
    @assert n == 2
    Φ₁_Aδ = Φ₁(Ax̃, δ)
    R̄err = Hyperrectangle(zeros(2), θ*δ)
    l̄ = abs.(inv(Φ₁_Aδ)) * θ * δ
    L̄ = Hyperrectangle(zeros(2), l̄)
    return R̄err, L̄
end

# let's assume n = 2 to simplify
function lagrange_remainder(f, Rlin, R̄err, x̃; n=2)
    @assert n == 2
    
    Hf₁ = [∂(f[1], (2, 0)) ∂(f[1], (1, 1));
           ∂(f[1], (1, 1)) ∂(f[1], (0, 2))]
    Hf₂ = [∂(f[2], (2, 0)) ∂(f[2], (1, 1));
           ∂(f[2], (1, 1)) ∂(f[2], (0, 2))]

    R̂lin = ConvexHullArray([Ri.X for Ri in Rlin.Xk]) ⊕ R̄err
    R̂lin_rect = overapproximate(R̂lin, Hyperrectangle)

    ξ = CH(Singleton(x̃), R̂lin_rect)
    ξ_rect = overapproximate(ξ, Hyperrectangle)
    ξ_box = convert(IntervalBox, ξ_rect)

    Hf₁_box = map(Hf_ij -> evaluate(Hf_ij, ξ_box), Hf₁)
    Hf₂_box = map(Hf_ij -> evaluate(Hf_ij, ξ_box), Hf₂)

    R̂lin_zono = convert(Zonotope, R̂lin_rect)

    γ = abs.(R̂lin_zono.center - x̃) + sum(abs.(R̂lin_zono.generators), dims=2)
    
    G = [sup.(abs.(Hf₁_box)), sup.(abs.(Hf₂_box))];
    li_zono = [(1/2 * transpose(γ) * G[i] * γ)[1, 1] for i in 1:n]
    L = Hyperrectangle(zeros(n), li_zono)
    return L
end

In [ ]:
function _add_chunk!(Rsets::Vector{HRECT}, Rlin, R̄err, t0)
    @inbounds for i in eachindex(Rlin.Xk)
        Ri = Rlin.Xk[i].X ⊕ R̄err
        Ri = overapproximate(Ri, Hyperrectangle)
        Ri = ReachSet(Ri, t0 + Rlin.Xk[i].t_start, t0 + Rlin.Xk[i].t_end)
        push!(Rsets[k], Ri)
    end
    return Rsets
end

function _add_chunk!(Rsets::Vector{ZONO}, k, Rlin, R̄err, t0)
    @inbounds for i in eachindex(Rlin.Xk)
        Ri = minkowski_sum(Rlin.Xk[i].X, convert(Zonotope, R̄err))
        Ri = ReachSet(Ri, t0 + Rlin.Xk[i].t_start, t0 + Rlin.Xk[i].t_end)
        push!(Rsets[k], Ri)
    end
    return Rsets
end

In [ ]:
using MathematicalSystems: AbstractContinuousSystem

const HRECT = ReachSet{Hyperrectangle{Float64}, Float64}
const ZONO = ReachSet{Zonotope{Float64}, Float64}

# TODO: generalize to the set type in the options 
_opC_return_type(opC::BFFPSV18) = Vector{HRECT}
_opC_return_type(opC::GLGM06) = Vector{ZONO}

"""
    LinearizedSystem{PT, VT} <: AbstractContinuousSystem

### Fields

- `𝑃`   -- linear initial value problem
- `x̃`   -- linearization point
- `dom` -- domain of validity of the linearization
"""
struct LinearizedSystem{PT, VT} <: AbstractContinuousSystem
    𝑃::PT # linearized IVP
    x̃::VT # lineariztion point
    dom::IntervalArithmetic.Interval{Float64}
end

## Main Algorithm

In [ ]:
# ====================
# Initialization
# ====================

TH = Hyperrectangle{Float64} # base set type

f = 𝑃.s.f # let's assume that 𝑃 is a BBCS and that 𝑂 are some options
n = statedim(𝑃) # state space dimension
Ω0 = 𝑃.x0 # initial set
T = 4 # time horizon
δ = 0.1 # step size for local time horizon
δcont = 0.05  # step size for continuous post
N = round(Int, T / δ) # number of sets for continuous post
θ = 𝑂_ASB08[:θ]
split_blocks = 𝑂_ASB08[:split_blocks]
# max_chunks = ? 

𝑂 = Options(:T=>T, :project_reachset=>true)
𝑂cont = Options(:δ=>δcont)

# set type of the final result
# the following stores a vector of vectors (VOA) each of hyperrectangular type
# let k be the integer that indicates the chunk being computed: [δ*(k-1), δk] for k = 1, ..., N
# ie. k = 0 => [0, δ] : we associate to it more than one flowpipe in general
#     k = 1 => [δ, 2δ] : similar for all other time intervals
#          . . .
#     k = N => [δ(N-1), δN]
#Rsets = VectorOfArray(Vector{Vector{TH}}(), (1, 1))
#typeof(Rsets)
Rsets = Vector{Vector{Vector{TH}}}(undef, N) # voy a tener 10 guachos

Qdot = []
push!(Qdot, Ω0) # queue of sets to be processed

In [ ]:
round(Int, T)

In [ ]:
# ====================
# Main loop
# ====================
T = round(Int, T)

for i in 1:T  # 1 : N   # index on the current chunk
    Q = deepcopy(Qdot) # update queue
    Qdot = []
    Ωi = []
    push!(Ωi, EmptySet{Float64}())
    Ωint = []
    
    Ωh = [] # UnionSetArray{Float64, <:LazySet}()
    push!(Ωh, EmptySet{Float64}())
    Ωhint = []
    
    # suppose that n is onocida
    
    while !isempty(Q)
        Ω̄ = pop!(Q)  # remove set from queue
        x̃, P = linearize(f, n, Ω0, δ)   # linearize(𝑃_curr, δ)
        println("starting chunk $i")
        Rlin = solve(P, 𝑂, op=BFFPSV18(𝑂cont))
        println("finished chunk $i")
        
        #Ωh = Rlin.Xk[end].X
        push!(Ωh, Rlin.Xk[end].X)

        Ωhint = [Rlin.Xk[i].X for i in 1:length(Rlin.Xk)]

        
        # compute set of admissible linearization errors
        R̄err, L̄ = admissible_error(P.s.A, δ, θ, n=2)

        
        # estimate Lagrange remainder
        L = lagrange_remainder(f, Rlin, R̄err, x̃, n=2)
        
        if L ⊆ L̄
            push!(Qdot, Ωh)
            push!(Ωi, Ωh) # really should take the set union here though
            push!(Ωint)
        else
            error("needs split!")
            push!(Q, split(Ω̄, split_blocks))
        end
    end
end

In [ ]:
plot*Qint

In [ ]:

# count number of continuous post chunks added to Rsets
nchunks = 0

# queue of problems to be processed
#𝑃_queue = Vector{typeof(𝑃)}()
#push!(𝑃_queue, 𝑃)

# queue of sets to be processed
Qp_queue = Vector{T}()
push!(Qp_queue, 𝑃)


In [ ]:
# ====================
# Main loop
# ====================

for i in 1:N
        
        
        #=
    while !isempty(𝑃_queue)
        println("length(𝑃_queue) = $(length(𝑃_queue))")
        𝑃_curr = pop!(𝑃_queue)
        k_curr = pop!(k_queue)

        # obtain linearized system
        Lin = linearize(𝑃_curr, δ)
        Plin = Lin.P
        
        # compute flowpipe of the linearized system
        Rlin = solve(Plinear.P, 𝑂cont, op=𝑂[:opC])

        # compute set of admissible linearization errors
        R̄err, L̄ = admissible_error(𝑃lin.s.A, δ, θ, n=2)
        L = lagrange_remainder(f, Rlin, R̄err, x̃, n=2)

         if !(L ⊆ L̄)
             println("splitting chunk $nchunks")
             𝑋₀split = split(𝑃lin.x0, split_blocks)

             for 𝑋₀i in 𝑋₀split
                 push!(𝑃_queue, IVP(𝑃.s, 𝑋₀i))
                 push!(k_queue, k_curr)
             end
         else
             nchunks += 1
             println("adding chunk $nchunks")
             _add_chunk!(Rsets[k_curr], Rlin, R̄err, δ * (k_curr - 1) )
             if δ * (k_curr + 1) < T
                push!(𝑃_queue, IVP(𝑃.s, Rsets[k_curr][end].X))
                push!(k_queue, k_curr + 1)
             end
         end
            
         if nchunks > max_chunks
             error("maximum number of chunks exceeded")
             #return ReachSolution(Rsets, 𝑂)
         end
            
        # inclusion test
        # . . . 
    end 
=#

## Older stuff

In [ ]:
# The structure that holds the flowpipe, Rsets, is a VectorOfArray
function post(𝑃::InitialValueProblem{<:BBCS}, 𝑂::Options)

    # ====================
    # Initialization
    # ====================
    
    # get nonlinear ODE
    f = 𝑃.s.f
    
    # unrwap commonly used options
    T, δ, δcont, θ = 𝑂[:T], 𝑂[:δ], 𝑂[:opC].options[:δ], 𝑂[:θ]
    N = T / δ
    split_blocks, max_chunks = 𝑂[:split_blocks], 𝑂[:max_chunks]
    
    # final result is stored in Rsets
    #T = _opC_return_type(𝑂[:opC])
    T = Hyperrectangle{Float64}
    Rsets = VectorOfArray(Vector{Vector{T}}(), (1, 1)) # vector de vectores de tipo T
    
    # count number of continuous post chunks added to Rsets
    nchunks = 0

    # queue of problems to be processed
    𝑃_queue = Vector{typeof(𝑃)}()
    push!(𝑃_queue, 𝑃)

    # queue of sets to be processed
    Qp_queue = Vector{T}()
    push!(Qp_queue, 𝑃)
    
    # integer for the chunk being computed: [δ*(k-1), δk]
    # ie. k = 0 => [0, δ]
    #     k = 1 => [δ, 2δ]
    #          . . .
    #     k = N => [δ(N-1), δN]
    #k_queue = Vector{Int}()
    #push!(k_queue, 1)

    # ====================
    # Main loop
    # ====================

    for i in 1:N
        
        
        #=
    while !isempty(𝑃_queue)
        println("length(𝑃_queue) = $(length(𝑃_queue))")
        𝑃_curr = pop!(𝑃_queue)
        k_curr = pop!(k_queue)

        # obtain linearized system
        Lin = linearize(𝑃_curr, δ)
        Plin = Lin.P
        
        # compute flowpipe of the linearized system
        Rlin = solve(Plinear.P, 𝑂cont, op=𝑂[:opC])

        # compute set of admissible linearization errors
        R̄err, L̄ = admissible_error(𝑃lin.s.A, δ, θ, n=2)
        L = lagrange_remainder(f, Rlin, R̄err, x̃, n=2)

         if !(L ⊆ L̄)
             println("splitting chunk $nchunks")
             𝑋₀split = split(𝑃lin.x0, split_blocks)

             for 𝑋₀i in 𝑋₀split
                 push!(𝑃_queue, IVP(𝑃.s, 𝑋₀i))
                 push!(k_queue, k_curr)
             end
         else
             nchunks += 1
             println("adding chunk $nchunks")
             _add_chunk!(Rsets[k_curr], Rlin, R̄err, δ * (k_curr - 1) )
             if δ * (k_curr + 1) < T
                push!(𝑃_queue, IVP(𝑃.s, Rsets[k_curr][end].X))
                push!(k_queue, k_curr + 1)
             end
         end
            
         if nchunks > max_chunks
             error("maximum number of chunks exceeded")
             #return ReachSolution(Rsets, 𝑂)
         end
            
        # inclusion test
        # . . . 
    end 
        
        =#

    println("total chunks = $nchunks")
    return Rsets
    #return ReachSolution(Rsets, 𝑂)
end

In [ ]:
T = _opC_return_type(BFFPSV18())
Rsets = VectorOfArray(Vector{Vector{T}}(), (1, 1))
push!(Rsets, Vector{HRECT}())

In [ ]:
typeof(Rsets)

In [ ]:
typeof(Rsets[1])

In [ ]:
@time Rsets = post(𝑃, 𝑂, 𝑂_ASB08);
#nsets = length(Rsol.Xk)

In [ ]:
maximum([Rsol.Xk[i].t_end for i in 1:nsets])

In [ ]:
plot(Rsol, xlab="x", ylab="y", alpha=.5, indices=1:500:nsets)
plot!(x->x, x->2.75, -3., 3., line=2, color="red", linestyle=:dash, legend=nothing)

In [ ]:
# check that specification holds
@assert all([ρ([0.0, 1.0], Rsol.Xk[i].X) < 2.75 for i in eachindex(Rsol.Xk)])

In [ ]:
maximum([ρ([0.0, 1.0], Rsol.Xk[i].X) for i in eachindex(Rsol.Xk)])

In [ ]:
any([Rsol.Xk[i].X ⊆ Rsol.Xk[1].X for i in 2:nsets])

### Other scripts

In [ ]:
#=
@time begin
# number of Taylor terms considered in the linearization
taylor_terms = 4

# define the working variables and fix the max order in the Taylor series expansion
x = set_variables("x", numvars=2, order=taylor_terms)

# define the ODE
f = Vector{TaylorN{Float64}}(undef, 2)
f[1] = x[2]
f[2] = x[2] * (1-x[1]^2) - x[1]

# define the initial-value problem
𝑋₀ = Hyperrectangle(low=[1.25, 2.35], high=[1.55, 2.45])
𝑆 = BlackBoxContinuousSystem(f, 2)
𝑃 = InitialValueProblem(𝑆, 𝑋₀)

# take the gradient of the vector field symbolically
#∇f = [gradient(f[i]) for i in 1:2]

# take the Jacobian matrix of the vector field symbolically
#Jf = [∂(f[1], (1, 0)) ∂(f[1], (0, 1));
#      ∂(f[2], (1, 0)) ∂(f[2], (0, 1))]

# algorithm-specific options
𝑂 = Options(:δ => 0.02, :δcont => 0.02/10, :max_order=>2, :θ=>fill(0.05, 2))

# unwrap options
δ = O[:δ]
θ = O[:θ]

# collection of flowpipes; the set type depends on add_chunk! and the continuous post
Rsets = Vector{ReachSet{Hyperrectangle{Float64}, Float64}}()
end
=#

In [ ]:
#=
@time begin
x̃, 𝑃lin = linearize(𝑃, δ)
end;
𝑋₀i = 𝑃lin.x0

# use zonotope-based continuous reach
@time begin
Rlin_zono = solve(𝑃lin, Options(:T=>O[:δ]), op=GLGM06(:δ=>O[:δcont], :max_order=>O[:max_order]))
end;

# use decomposition-based continuous reach
@time begin
Rlin_box = solve(𝑃lin, Options(:T=>O[:δ]), op=BFFPSV18(:δ=>O[:δcont]))
end;

plot(Rlin_zono, alpha=.5)
plot!(Rlin_box, alpha=.5, xlab="x", ylab="y")

Rlin = Rlin_zono  # decide which continuoust post to use

@time begin
R̄err, L̄ = admissible_error(𝑃lin.s.A, δ, θ; n=2)
L = lagrange_remainder(f, Rlin, R̄err, x̃; n=2)
end
=#